In [13]:
import MetaTrader5 as mt5
from time import sleep
from datetime import datetime
import pytz

In [15]:

now= datetime.now() #(yr,M, D, hr, min, sec)
print(f'now is {now}')

formatted_date = now.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_date)

# print('\n')
# # Get the current time from the broker
# broker_time = mt5.time()
# broker_datetime = datetime.fromtimestamp(broker_time, pytz.utc)
# print("Broker datetime:", broker_datetime)


import datetime
import pytz

# Set the timezone to UTC
utc = pytz.UTC

# Get the current time in UTC
current_time = datetime.datetime.now(utc)

print(f'current_time in utc: {current_time} ')


now is 2025-01-14 05:31:33.154915
2025-01-14 05:31:33
current_time in utc: 2025-01-14 10:31:33.155904+00:00 


"When creating the 'datetime' object, Python uses the local time zone, while MetaTrader 5 stores tick and bar open time in UTC time zone (without the shift). Therefore, 'datetime' should be created in UTC time for executing functions that use time. The data obtained from MetaTrader 5 have UTC time, but Python applies the local time shift again when trying to print them."


In [ ]:
# import credentials from environments
from dotenv import load_dotenv
import os



In [2]:

if not mt5.initialize(): 
    print( 'initialization failed')
    quit() 
else: print('initialization success') 



login = 192918303
password = "Demo@1234"
server = "Exness-MT5Trial"

# mt5.login(login, password, server)


if not mt5.login(login, password, server):
    print("Failed to connect to the server")
else:
    print("Connected to the server successfully")
    print(mt5.account_info())


initialization success
Connected to the server successfully
AccountInfo(login=192918303, trade_mode=0, leverage=200, limit_orders=1024, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=9952.36, credit=0.0, profit=0.0, equity=9952.36, margin=0.0, margin_free=9952.36, margin_level=0.0, margin_so_call=60.0, margin_so_so=0.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='DemoBotTester', server='Exness-MT5Trial', currency='USD', company='Exness Technologies Ltd')


### Send market order

In [31]:

symbol = 'EURUSDm'
volume = 0.1
action = mt5.TRADE_ACTION_DEAL
order_type = mt5.ORDER_TYPE_BUY

stop_loss = 0  # set to 0.0 if you don't want SL
take_profit = 0  # set to 0.0 if you don't want TP

def get_market_price(symbol, type):
    if type == mt5.ORDER_TYPE_BUY: # 1 or true for buy
        return mt5.symbol_info(symbol).ask
    elif type == mt5.ORDER_TYPE_SELL: # 0 or false for sell
        return mt5.symbol_info(symbol).bid

request = {
    "action": action,
    "symbol": symbol,
    "volume": volume,  # float
    "type": mt5.ORDER_TYPE_BUY,
    "price": get_market_price(symbol, 0),
    "sl": stop_loss,  # float
    "tp": take_profit,  # float
    "deviation": 20,
    "magic": 0, # magic when you want to run different strategy and trades at the same time
    "comment": "python market order",
    "type_time": mt5.ORDER_TIME_GTC, #The order stays in the queue until it is manually canceled
    "type_filling": mt5.ORDER_FILLING_IOC,  # some brokers accept mt5.ORDER_FILLING_FOK only
}

res = mt5.order_send(request)
res

# i realised i cant send i live order from jupyter notebook except for limit orders

## modify SL/TP

In [33]:
stop_loss = 0.9  # set to 0.0 if you don't want SL
take_profit = 1.2  # set to 0.0 if you don't want TP

positions = mt5.positions_get()
print('open positions', positions)

# Working with 1st position in the list and closing it
pos1 = positions[0]

request = {
    'action': mt5.TRADE_ACTION_SLTP,# key instrument here
    'position': pos1.ticket,
    'sl': stop_loss,
    'tp': take_profit
}

res = mt5.order_send(request)
res

# order change works from notebook

open positions (TradePosition(ticket=1332185369, time=1736852475, time_msc=1736852475788, time_update=1736852475, time_update_msc=1736852475788, type=0, magic=0, identifier=1332185369, reason=0, volume=0.01, price_open=1.0259200000000002, sl=0.0, tp=0.0, price_current=1.02583, swap=0.0, profit=-0.09, symbol='EURUSDm', comment='', external_id=''),)


OrderSendResult(retcode=10009, deal=0, order=0, volume=0.01, price=1.02583, bid=1.02583, ask=1.0259200000000002, comment='', request_id=3376697456, retcode_external=0, request=TradeRequest(action=6, magic=0, order=0, symbol='', volume=0.0, price=0.0, stoplimit=0.0, sl=0.9, tp=1.2, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=1332185369, position_by=0))

## close position


In [34]:
positions = mt5.positions_get()
print('open positions', positions)

# Working with 1st position in the list and closing it
pos1 = positions[0]

def reverse_type(type):
    # to close a buy positions, you must perform a sell position and vice versa
    if type == mt5.ORDER_TYPE_BUY:
        return mt5.ORDER_TYPE_SELL
    elif type == mt5.ORDER_TYPE_SELL:
        return mt5.ORDER_TYPE_BUY


def get_close_price(symbol, type):
    if type == mt5.ORDER_TYPE_BUY:
        return mt5.symbol_info(symbol).bid
    elif type == mt5.ORDER_TYPE_SELL:
        return mt5.symbol_info(symbol).ask

request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "position": pos1.ticket,
    "symbol": pos1.symbol,
    "volume": pos1.volume,
    "type": reverse_type(pos1.type),
    "price":get_close_price(pos1.symbol, pos1.type),
    "deviation": 20,
    "magic": 0,
    "comment": "python close order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,  # some brokers accept mt5.ORDER_FILLING_FOK only
}

res = mt5.order_send(request)
res

open positions (TradePosition(ticket=1332185369, time=1736852475, time_msc=1736852475788, time_update=1736852475, time_update_msc=1736852475788, type=0, magic=0, identifier=1332185369, reason=0, volume=0.01, price_open=1.0259200000000002, sl=0.9, tp=1.2, price_current=1.02576, swap=0.0, profit=-0.16, symbol='EURUSDm', comment='', external_id=''),)


OrderSendResult(retcode=10009, deal=1013141510, order=1332187646, volume=0.01, price=1.02576, bid=1.02576, ask=1.0258500000000002, comment='python close ord', request_id=3376697457, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='EURUSDm', volume=0.01, price=1.02576, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=1, type_time=0, expiration=0, comment='python close order', position=1332185369, position_by=0))

## partial close position

In [39]:
perc_close = 0.5 # percentage to close

positions = mt5.positions_get()
print('open positions', positions)

# Working with 1st position in the list and closing it
pos1 = positions[0]

def reverse_type(type):
    # to close a buy positions, you must perform a sell position and vice versa
    if type == 0:
        return 1
    elif type == 1:
        return 0


def get_close_price(symbol, type):
    if type == 0:
        return mt5.symbol_info(symbol).bid
    elif type == 1:
        return mt5.symbol_info(symbol).ask

request = {
    "action": mt5.TRADE_ACTION_DEAL,
    "position": pos1.ticket,
    "symbol": pos1.symbol,
    "volume": round(pos1.volume * perc_close, 2),  # closing only half
    "type": reverse_type(pos1.type),
    "price":get_close_price(pos1.symbol, pos1.type),
    "deviation": 20,
    "magic": 0,
    "comment": "python partial close order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,  # some brokers accept mt5.ORDER_FILLING_FOK only
}

res = mt5.order_send(request)
res

open positions (TradePosition(ticket=1332190605, time=1736852611, time_msc=1736852611893, time_update=1736852611, time_update_msc=1736852611893, type=0, magic=0, identifier=1332190605, reason=0, volume=1.0, price_open=1.0258500000000002, sl=0.0, tp=0.0, price_current=1.02575, swap=0.0, profit=-10.0, symbol='EURUSDm', comment='', external_id=''),)


OrderSendResult(retcode=10009, deal=1013144731, order=1332190834, volume=0.5, price=1.0257500000000002, bid=1.0257500000000002, ask=1.02584, comment='python partial c', request_id=3376697463, retcode_external=0, request=TradeRequest(action=1, magic=0, order=0, symbol='EURUSDm', volume=0.5, price=1.02575, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=1, type_time=0, expiration=0, comment='python partial close order', position=1332190605, position_by=0))

# limit orders


In [26]:
# limit orders

symbol = 'EURUSDm'
volume = 0.1

action = mt5.TRADE_ACTION_PENDING #key instrument here
order_type = mt5.ORDER_TYPE_BUY_LIMIT
limit_price = 1.0

stop_loss = 0.9  # set to 0.0 if you don't want SL
take_profit = 1.2  # set to 0.0 if you don't want TP

request = {
    "action": action, #key instrument
    "symbol": symbol,
    "volume": volume,  # float
    "type": order_type,
    "price": limit_price,
    "sl": stop_loss,  # float
    "tp": take_profit,  # float
    "deviation": 20,
    "magic": 0,
    "comment": "python market order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,  # some brokers accept mt5.ORDER_FILLING_FOK only
}

res = mt5.order_send(request)
res

OrderSendResult(retcode=10009, deal=0, order=1332172290, volume=0.1, price=0.0, bid=1.0258200000000002, ask=1.02591, comment='python market or', request_id=3376697451, retcode_external=0, request=TradeRequest(action=5, magic=0, order=0, symbol='EURUSDm', volume=0.1, price=1.0, stoplimit=0.0, sl=0.9, tp=1.2, deviation=20, type=2, type_filling=1, type_time=0, expiration=0, comment='python market order', position=0, position_by=0))

## stop orders

In [45]:
# stop orders

symbol = 'EURUSDm'
volume = 0.1

action = mt5.TRADE_ACTION_PENDING
order_type = mt5.ORDER_TYPE_BUY_STOP
stop_price = 1.2

stop_loss = 1.0  # set to 0.0 if you don't want SL
take_profit = 1.3  # set to 0.0 if you don't want TP

request = {
    "action": action,
    "symbol": symbol,
    "volume": volume,  # float
    "type": order_type,
    "price": stop_price,
    "sl": stop_loss,  # float
    "tp": take_profit,  # float
    "deviation": 20,
    "magic": 0,
    "comment": "python market order",
    "type_time": mt5.ORDER_TIME_GTC,
    "type_filling": mt5.ORDER_FILLING_IOC,  # some brokers accept mt5.ORDER_FILLING_FOK only
}

res = mt5.order_send(request)
res

OrderSendResult(retcode=10013, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid request', request_id=0, retcode_external=0, request=TradeRequest(action=5, magic=0, order=0, symbol='EURUSD', volume=0.1, price=1.2, stoplimit=0.0, sl=1.0, tp=1.3, deviation=20, type=4, type_filling=1, type_time=0, expiration=0, comment='python market order', position=0, position_by=0))

## Remove Pending Order

In [52]:
pending_orders = mt5.orders_get()
print('open pending orders', pending_orders)

order1 = pending_orders[0]

request = {
    'action': mt5.TRADE_ACTION_REMOVE,
    'order': order1.ticket
}

mt5.order_send(request)

open pending orders (TradeOrder(ticket=1332223581, time_setup=1736853344, time_setup_msc=1736853344955, time_done=0, time_done_msc=0, time_expiration=0, type=2, type_time=0, type_filling=2, state=1, magic=0, position_id=0, position_by_id=0, reason=0, volume_initial=1.0, volume_current=1.0, price_open=1.02525, sl=0.0, tp=0.0, price_current=1.02588, price_stoplimit=0.0, symbol='EURUSDm', comment='', external_id=''),)


OrderSendResult(retcode=10009, deal=0, order=1332223581, volume=1.0, price=1.02525, bid=1.02579, ask=1.0258800000000001, comment='', request_id=3376697470, retcode_external=0, request=TradeRequest(action=8, magic=0, order=1332223581, symbol='', volume=0.0, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=0, type=0, type_filling=0, type_time=0, expiration=0, comment='', position=0, position_by=0))